In [86]:
from transformers import pipeline
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings
from langchain_huggingface import HuggingFacePipeline

from langchain_community.document_loaders import ImageCaptionLoader

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.document_loaders.csv_loader import CSVLoader

import pandas as pd


In [88]:
df = pd.read_csv("/Users/gabrielaburaglia/Desktop/fashion_data.csv")
df.head(10)

,image,description,display name,category
0,3238.jpg,"Round toed, black sports shoes with red accent...",Puma Men Black 65CC Lo Ducati Sports Shoes,Sports Shoes
1,43044.jpg,Style Note Built with the breathability and ze...,Nike Men Charcoal Grey Shorts,Shorts
2,54018.jpg,Teal handbag that has stitch detailing with a...,Kiara Women Teal Handbag,Handbags
3,8141.jpg,"Perfectly stylish, this fastrack analog wrist ...",Fastrack Women Freestyle Sports Analog Steel B...,Watches
4,22245.jpg,These id mid-top chukka shoes add a fresh spin...,ID Men Brown Casual Shoes,Casual Shoes
5,19572.jpg,Composition Red round neck t-shirt made of 100...,United Colors of Benetton Men Printed Red T-shirt,Tshirts
6,25305.jpg,"Crafted for the urban style maven, this brown ...",Lino Perros Women Leather Brown Wallet,Wallets
7,16704.jpg,Composition Navy blue round neck sleeveless t-...,Fila Men High Density Navy Blue Tshirt,Tshirts
8,16509.jpg,Composition Dark blue jeans made of 100% cotto...,Locomotive Men Washed Blue Jeans,Jeans
9,24661.jpg,"Made of 82% cotton, 16% nylon and 2% elastane ...",ADIDAS Originals Men White Casual Socks,Socks


In [89]:
import os
dir = os.getcwd()
db_dir = os.path.join(dir,"chroma_db")
print(db_dir)

/Users/gabrielaburaglia/Documents/GitHub/closet_care_buddy/chroma_db


In [90]:
loader = CSVLoader(file_path="/Users/gabrielaburaglia/Desktop/fashion_data.csv")

data = loader.load()

#Split the document into chunks using text splitters 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

print("Document chunk info:\n")
print(f"Number of document chunks: {len(chunks)}")
print(f"Sample chunk: \n{chunks[3].page_content}\n")

Document chunk info:

Number of document chunks: 56038
Sample chunk: 
image: 8141.jpg
description: Perfectly stylish, this fastrack analog wrist watch is a great buy for those who enjoy performance and style. With excellent touches and comfort, this watch makes for an excellent daily wear. Case diameter : 30 mm Case thickness : 5 mm Dial shape : Square   Plastic case with a fixed bezel for added durability, style and comfort  Leather strap with stitch detailing, brand name printed on the inner part and a tang clasp  Silver dial in a radiating pattern with minute marking in pink from 6 o&  rsquo clock to 12 o&  rsquo clock position, brand name at 3 o&  rsquo clock position and brand logo at 12 o&  rsquo clock position  Quartz movement of time display with carved out hour and minute hands  Screw to reset time  Solid case back made of stainless steel for enhanced durability  Water resistance at 30 meters, allowing you to use it even when swimming
display name: Fastrack Women Freestyle Spo

In [91]:
#create embeddings using openAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
#store the embeddings and chunks into Chroma DB
Chroma.from_documents(documents=chunks[:int(len(chunks)/2)], embedding=embeddings, persist_directory=db_dir) 

KeyboardInterrupt: 

In [ ]:
Chroma.from_documents(documents=chunks[int(len(chunks)/2):], embedding=embeddings, persist_directory=db_dir) 